In [2]:
import pandas as pd
import numpy as np
import datetime
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib.pylab import rcParams
import matplotlib
import folium
import warnings
import matplotlib.ticker as ticker
from sklearn.model_selection import TimeSeriesSplit
import pmdarima as pm
%matplotlib inline

In [5]:
# import data from CA, CT, TX, FL, NY, NJ 
ca = pd.read_csv('data/ca_ev_registrations_public.csv')
nj = pd.read_csv('data/nj_ev_registrations_public.csv')
ny = pd.read_csv('data/ny_ev_registrations_public.csv')
tx = pd.read_csv('data/tx_ev_registrations_public.csv')
fl = pd.read_csv('data/fl_ev_registrations_public.csv')
ct = pd.read_csv('data/ct_ev_registrations_public.csv')

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
ca.info()
# 2542443 entries in CA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542443 entries, 0 to 2542442
Data columns (total 9 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Vehicle ID                    object 
 1   County GEOID                  object 
 2   Registration Valid Date       object 
 3   DMV ID                        int64  
 4   DMV Snapshot                  object 
 5   Registration Expiration Date  float64
 6   State Abbreviation            object 
 7   Geography                     object 
 8   Vehicle Name                  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 174.6+ MB


In [9]:
nj.info()
# NJ has 309872 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309872 entries, 0 to 309871
Data columns (total 10 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   DMV ID                        309872 non-null  int64  
 1   DMV Snapshot (Date)           309872 non-null  object 
 2   Registration Valid Date       309872 non-null  object 
 3   ZIP Code                      309872 non-null  object 
 4   VIN Prefix                    309872 non-null  object 
 5   VIN Model Year                309872 non-null  object 
 6   Registration Expiration Date  0 non-null       float64
 7   Make                          309872 non-null  object 
 8   Model                         309872 non-null  object 
 9   Model Year                    309872 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 23.6+ MB


In [10]:
ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118179 entries, 0 to 3118178
Data columns (total 10 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   DMV ID                        int64 
 1   DMV Snapshot                  object
 2   ZIP Code                      int64 
 3   State                         object
 4   Registration Valid Date       object
 5   Registration Expiration Date  object
 6   VIN Prefix                    object
 7   VIN Model Year                object
 8   Vehicle Name                  object
 9   Technology                    object
dtypes: int64(2), object(8)
memory usage: 237.9+ MB


In [11]:
tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1446588 entries, 0 to 1446587
Data columns (total 12 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Vehicle ID                    1446588 non-null  object 
 1   ZIP Code                      1446588 non-null  object 
 2   Registration Valid Date       1446588 non-null  object 
 3   VIN Prefix                    1446588 non-null  object 
 4   DMV_ID                        1446588 non-null  int64  
 5   DMV ID Complete               1446588 non-null  object 
 6   DMV Snapshot                  1446588 non-null  object 
 7   Registration Expiration Date  0 non-null        float64
 8   State Abbreviation            1446588 non-null  object 
 9   Geography                     1446588 non-null  object 
 10  Vehicle Name                  1446588 non-null  object 
 11  Technology                    1446588 non-null  object 
dtypes: float64(1), int64(1), obj

In [12]:
ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11439 entries, 0 to 11438
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   DMV ID                        11439 non-null  int64 
 1   DMV Snapshot (Date)           11439 non-null  object
 2   ZIP Code                      11439 non-null  int64 
 3   Registration Valid Date       11439 non-null  object
 4   Registration Expiration Date  11439 non-null  object
 5   VIN Prefix                    11439 non-null  object
 6   VIN Model Year                11439 non-null  object
 7   Make                          11439 non-null  object
 8   Model                         11439 non-null  object
 9   Model Year                    11439 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 893.8+ KB


In [14]:
fl.head()

,DMV ID,DMV Snapshot (Date),County,Vehicle Name,Registration Valid Date,Registration Expiration Date,Technology
0,1,Registration Data from FPL (6/30/2018),Dade,Tesla Model X,6/30/2018,NaN,BEV
1,1,Registration Data from FPL (6/30/2018),Dade,Tesla Model X,6/30/2018,NaN,BEV
2,1,Registration Data from FPL (6/30/2018),Dade,Tesla Model X,6/30/2018,NaN,BEV
3,1,Registration Data from FPL (6/30/2018),Dade,Tesla Model X,6/30/2018,NaN,BEV
4,1,Registration Data from FPL (6/30/2018),Dade,Tesla Model X,6/30/2018,NaN,BEV
